## Парсер Яндекс Музыки

In [ ]:
#pip install requests beautifulsoup4

* get_image_url(artist_image_element):  
Извлекает URL изображения артиста из HTML-элемента. Преобразует исходный URL в формат, который возвращает изображение размером 200x200 пикселей.  
* get_similar(url):  
Получает информацию об артисте и списке похожих артистов с заданной страницы Яндекс.Музыки. Извлекает имя артиста, URL изображения, ID и аналогичную информацию для похожих артистов.  
* get_artist_urls_from_genre_page(genre_url):  
Извлекает URL-адреса всех артистов с заданной страницы жанра на Яндекс.Музыке. Формирует URL для страницы "похожие артисты" каждого исполнителя.  
* Основной блок кода:  
Определяет список URL-адресов страниц жанров для обработки. Затем проходит по каждому URL, получает список артистов, собирает данные о каждом артисте и его похожих исполнителях, и сохраняет эту информацию в файл. Избегает дублирования данных и добавляет случайные задержки между запросами.  

In [11]:
import requests
from bs4 import BeautifulSoup
import random
import time

def get_image_url(artist_image_element):
    if artist_image_element and 'src' in artist_image_element.attrs:
        parts = artist_image_element['src'].split('/get-music-content/')
        if len(parts) > 1 and len(parts[1].split('/')) >= 2:
            artist_id, prefix = parts[1].split('/')[0:2]
            return f'https://avatars.yandex.net/get-music-content/{artist_id}/{prefix}/200x200'
    return None


def get_similar(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    artist_name_element = soup.find('h1', class_='page-artist__title')
    if not artist_name_element:
        print("капча")
        return None

    artist_name = artist_name_element.text.strip()
    artist_image_element = soup.find('div', class_='artist-pics').find('img', class_='artist-pics__pic')
    artist_image_url = get_image_url(artist_image_element)
    artist_id = url.split('/artist/')[1].split('/')[0]

    similar_artists = []
    similar_artists_block = soup.find('div', class_='page-artist__artists')
    if similar_artists_block:
        for artist_element in similar_artists_block.find_all('div', class_='artist__content'):
            similar_artist_name = artist_element.find('div', class_='artist__name').text.strip()
            similar_artist_image_element = artist_element.find('div', class_='artist__cover').find('img', class_='artist-pics__pic')
            similar_artist_image_url = get_image_url(similar_artist_image_element)

            similar_artist_link = artist_element.find('a', class_='d-link', href=True)
            similar_artist_id = similar_artist_link['href'].split('/artist/')[1].split('/')[0] if similar_artist_link else None

            similar_artists.append(
                {"artist_name": similar_artist_name, "artist_image_url": similar_artist_image_url, "artist_id": similar_artist_id}
            )

    artist_data = {
        "artist_name": artist_name,
        "artist_image_url": artist_image_url,
        "artist_id": artist_id,
        "similar_artists": similar_artists
    }
    return artist_data


def get_artist_urls_from_genre_page(genre_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(genre_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    artist_urls = []
    artist_elements = soup.find_all('a', class_='d-link deco-link', href=True)
    for artist_element in artist_elements:
        href = artist_element['href']
        if '/artist/' in href:
            artist_id = href.split('/artist/')[1].split('/')[0]
            similar_artist_url = f'https://music.yandex.ru/artist/{artist_id}/similar'
            artist_urls.append(similar_artist_url)
    return artist_urls


genre_urls = [
    'https://music.yandex.ru/genre/русский%20рок/artists?page=1',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=2',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=3',

# ссылки, откуда надо парсить
    
]

with open('artist_data.txt', 'a+', encoding='utf-8') as file:
    for genre_url in genre_urls:
        artist_urls = get_artist_urls_from_genre_page(genre_url)
        for artist_url in artist_urls:
            artist_data = get_similar(artist_url)
            if artist_data:
                artist_id = artist_data['artist_id']

                file.seek(0)

                already_exists = False
                for line in file:
                    line_data = eval(line)
                    if line_data['artist_id'] == artist_id:
                        already_exists = True
                        break

                if not already_exists:
                    file.write(str(artist_data) + '\n')
                    print(artist_data)
                    print("-" * 20)

                time.sleep(random.uniform(5, 10))

## Пример

In [8]:
artist_data = get_similar("https://music.yandex.ru/artist/41119")
if artist_data:
    print(str(artist_data))

{'artist_name': 'Dino MC47', 'artist_image_url': 'https://avatars.yandex.net/get-music-content/5853241/3a95d451.p.41119/200x200', 'artist_id': '41119', 'similar_artists': [{'artist_name': 'ST1M', 'artist_image_url': 'https://avatars.yandex.net/get-music-content/9867886/801e1546.p.168660/200x200', 'artist_id': '168660'}, {'artist_name': 'Лигалайз', 'artist_image_url': 'https://avatars.yandex.net/get-music-content/3318009/06d55023.p.41186/200x200', 'artist_id': '41186'}, {'artist_name': 'Руставели', 'artist_image_url': 'https://avatars.yandex.net/get-music-content/117546/c3a0319b.p.233942/200x200', 'artist_id': '233942'}, {'artist_name': 'NTL', 'artist_image_url': 'https://avatars.yandex.net/get-music-content/5502420/af701718.p.4150953/200x200', 'artist_id': '4150953'}, {'artist_name': 'Shot', 'artist_image_url': 'https://avatars.yandex.net/get-music-content/6021799/80922bfe.a.22285044-1/200x200', 'artist_id': '2751847'}, {'artist_name': 'Johnyboy', 'artist_image_url': 'https://avatars.y

## Структура сайта

In [1]:
# def fetch_page_html(url):
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }

#     response = requests.get(url, headers=headers)
#     if response.status_code == 200:
#         return response.text
#     else:
#         raise Exception(f"Failed {url}, status code {response.status_code}")

# def analyze_html_structure(html):
#     soup = BeautifulSoup(html, 'html.parser')
#     print(soup.prettify())

#url = 'https://music.yandex.ru/artist/41191/similar'
#html = fetch_page_html(url)
#analyze_html_structure(html)

In [6]:
#url = 'https://music.yandex.ru/genre/местная%20инди-музыка/artists'
#html = fetch_page_html(url)
#analyze_html_structure(html)


In [ ]:
          # <div class="artist__name deco-typo typo-main" title="Монеточка">
          #  <span class="d-artists d-artists__expanded" data-b="">
          #   <a class="d-link deco-link" data-b="" href="/artist/4623000" title="Монеточка">
          #    Монеточка
          #   </a>
          #  </span>
          # </div>

In [ ]:
# спарсено в artist_data.txt
--- рок
    'https://music.yandex.ru/genre/русский%20рок/artists',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=1',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=2',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=3',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=4',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=5',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=6',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=7'
    'https://music.yandex.ru/genre/русский%20рок/artists?page=8',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=9',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=10',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=11',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=12',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=13',
    'https://music.yandex.ru/genre/русский%20рок/artists?page=14',
    

--- инди
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=1',
   'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=2',
   'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=3',
   'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=4',
   'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=5',
   'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=6',
   'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=7'
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=8',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=9',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=10',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=11',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=12',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=13',
    'https://music.yandex.ru/genre/местная%20инди-музыка/artists?page=14'


--- поп
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=1',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=2',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=3',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=4',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=5',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=6',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=7',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=8',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=9',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=10',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=11',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=12',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=13',
    'https://music.yandex.ru/genre/русская%20поп-музыка/artists?page=14'

--- рэп
    'https://music.yandex.ru/genre/русский%20рэп/artists',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=1',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=2',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=3',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=4',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=5',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=6',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=7'
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=8',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=9',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=10'
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=11'
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=12',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=13',
    'https://music.yandex.ru/genre/русский%20рэп/artists?page=14'


--- эстрада
    'https://music.yandex.ru/genre/русская%20эстрада/artists',
    'https://music.yandex.ru/genre/русская%20эстрада/artists?page=1',
    'https://music.yandex.ru/genre/русская%20эстрада/artists?page=2',
    'https://music.yandex.ru/genre/русская%20эстрада/artists?page=3',
    'https://music.yandex.ru/genre/русская%20эстрада/artists?page=4',
    'https://music.yandex.ru/genre/русская%20эстрада/artists?page=5',
    'https://music.yandex.ru/genre/русская%20эстрада/artists?page=6',
 

In [ ]:
# with open('artist_data.txt', 'r', encoding='utf-8') as file:
#     print(file.read())